In [1]:
import pandas as pd

import preprocessing as pp
import model
from train import train_model
from predict import predict

In [2]:
# Data loading
df_store = pd.read_csv('../data/store.csv')
df_train = pd.read_csv('../data/train.csv', low_memory=False)

## General pipeline test

### Model selection (Time-split cross-validation)

In [3]:
from sklearn.model_selection import TimeSeriesSplit
from train import rmse, r2

train = df_train.copy().iloc[::-1]
train.Date = pd.to_datetime(train.Date)

n_splits = 10
tscv = TimeSeriesSplit(n_splits=n_splits)

reg_model = model.Regressor()
rmse_scores = []
r2_scores = []

date_grouping = train.groupby(train.Date)['Store']
date_list = [g[0] for g in list(date_grouping)[:]]
for train_index, test_index in tscv.split(date_grouping):
    train_dates = [date_list[train_index[0]], date_list[train_index[-1]]]
    test_dates = [date_list[test_index[0]], date_list[test_index[-1]]]
    train_mask = (train.Date >= train_dates[0]) & (train.Date <= train_dates[1])
    test_mask = (train.Date >= test_dates[0]) & (train.Date <= test_dates[1])
    
    train.loc[test_mask]
    
    # Train and test sets
    X_train, y_train = pp.Preprocessor().transform(df_store, train.loc[train_mask])
    X_test, y_test = pp.Preprocessor().transform(df_store, train.loc[test_mask])
    
    # Dummy variables can induce differences in the schemas
    missing_test = set(X_train.columns) - set(X_test.columns)
    missing_train = set(X_test.columns) - set(X_train.columns)
    for c in missing_test:
        X_test[c] = 0
    for c in missing_train:
        X_train[c] = 0
    # Reorder to match columns order in train and test
    X_test = X_test[X_train.columns]
    
    # Model fitting on training set
    train_model(reg_model, X_train, y_train)

    # Scoring on test set
    y_pred = reg_model.predict(X_test)
    rmse_scores.append(rmse(y_test, y_pred))
    r2_scores.append(r2(y_test, y_pred))
        
# Final display
for i in range(n_splits):
    print("FOLD " + str(i + 1) + ": " + "RSME = " + str(rmse_scores[i]) + 
      " | R² = " + str(r2_scores[i]))

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Anaconda3\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


FOLD 1: RSME = 2389.9576481734425 | R² = 0.6260729929867368
FOLD 2: RSME = 2378.673278997117 | R² = 0.6382162678920918
